In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



def load_data(path):
    data = pd.read_csv(path)
    return data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



def load_data(path):
    data = pd.read_csv(path)
    return data

new_df=load_data("/content/drive/MyDrive/datathon/purelyData.csv")
y=new_df["Öbek İsmi"]
X=new_df.drop(["Öbek İsmi"],axis=1)
X = X.drop(X.columns[0], axis=1)

def models(X,y, scoring="roc_auc"):
  classifiers=[
               ("RF", RandomForestClassifier()),


               ]
  for name, classifier in classifiers:
        cv_results = cross_validate(classifier, X, y, cv=5, scoring=scoring)



print(models(X,y,scoring="accuracy"))



rf_params = {"max_depth": [1, 30, None],

             "max_features": [5, 20, "auto"],
             "min_samples_split": [15, 40],
             "n_estimators": [200, 375]}



classifiers = [
               ("RF", RandomForestClassifier(), rf_params)
               ]

from sklearn.model_selection import GridSearchCV


def hyperparameter_optimization(X, y, classifiers, cv=5, scoring="roc_auc"):
    # Fonksiyon kodu burada devam eder...

    print("Hyperparameter Optimization....")

    top_models = []  # Boş bir liste en iyi modelleri saklamak için

    for name, classifier, params in classifiers:

        cv_results = cross_validate(classifier, X, y, cv=cv, scoring=scoring)


        gs_best = GridSearchCV(classifier, params, cv=cv, n_jobs=-1, verbose=False).fit(X, y)
        final_model = classifier.set_params(**gs_best.best_params_)

        cv_results = cross_validate(final_model, X, y, cv=cv, scoring=scoring)


        # Modelleri performanslarına göre sırala ve en iyi 3'ü sakla
        top_models.append((name, final_model, cv_results['test_score'].mean()))
        top_models.sort(key=lambda x: x[2], reverse=True)
        top_models = top_models[:3]



    return  top_models

top_models = hyperparameter_optimization(X, y, classifiers, scoring="accuracy")

# En iyi 3 modeli yazdır
print("Top 3 Models:")
for idx, (name, model, score) in enumerate(top_models):
    print(f"{idx + 1}. {name}: {score:.4f}")


None
Hyperparameter Optimization....
Top 3 Models:
1. RF: 0.9544


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
import numpy as np


new_df=load_data("/content/drive/MyDrive/datathon/purelyData.csv")
y=new_df["Öbek İsmi"]
X=new_df.drop(["Öbek İsmi"],axis=1)
X = X.drop(X.columns[0], axis=1)


# Define the hyperparameter grid
rf_params = {
    "max_depth": [None, 5, 10, 15],  # Include more depth options
    "max_features": ["auto", "sqrt"],
    "min_samples_split": [ 5, 10, 20],  # Include more split options
    "min_samples_leaf": [1, 2, 4,6],  # Include more leaf options
    "n_estimators": [100, 300,350] }

classifiers = [("RF", RandomForestClassifier(random_state=42), rf_params)]

def hyperparameter_optimization(X, y, classifiers, cv=5, scoring="accuracy"):
    print("Hyperparameter Optimization....")
    top_models = []  # To store the top models

    for name, classifier, params in classifiers:
        gs = GridSearchCV(classifier, params, cv=cv, n_jobs=-1, scoring=scoring)
        gs.fit(X, y)
        best_model = gs.best_estimator_

        cv_scores = cross_val_score(best_model, X, y, cv=cv, scoring=scoring)
        avg_score = np.mean(cv_scores)

        # Store the top models based on average CV score
        top_models.append((name, best_model, avg_score))
        top_models.sort(key=lambda x: x[2], reverse=True)
        top_models = top_models[:3]

    return top_models

# Assuming X and y are your dataset and labels
top_models = hyperparameter_optimization(X, y, classifiers, scoring="accuracy")

# Print the top 3 models
print("Top 3 Models:")
for idx, (name, model, score) in enumerate(top_models):
    print(f"{idx + 1}. {name}: {score:.4f}")





Hyperparameter Optimization....


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Top 3 Models:
1. RF: 0.9548


In [ ]:
test_data=load_data("/content/drive/MyDrive/datathon/test.csv")
test_data=test_data.drop(test_data.columns[0],axis=1)
model= top_models[0][1]

predictions= model.predict(test_data)
predictions_df = pd.DataFrame(predictions + 1, columns=["Öbek İsmi"], index=test_data.index)
predictions_df["Öbek İsmi"] = "obek_"+predictions_df["Öbek İsmi"].astype(str)
predictions_df.index.name = "id"


csv_filename = 'predictionsNew.csv'  # Kaydetmek istediğiniz dosya adını burada belirtin
predictions_df.to_csv(csv_filename, index=True)